## Translate GOLD study, project, and biosample data into json.
The notebooks demostrates how to translate study, project, and biosample data from the GOLD database into json that conforms with the [NMDC schema](https://github.com/microbiomedata/nmdc-metadata/blob/schema-draft/README.md).  
Before doing the translation it is important that you have an up to date `nmdc.py` file in the `lib` directory.  

The python modules for running the notebook are in the `requirements.txt` file.  

In [1]:
import json
import pandas as pds
import jsonasobj
import lib.nmdc as nmdc
import lib.data_operations as dop
from pandasql import sqldf

def pysqldf(q):
    return sqldf(q, globals())

## Load tables (i.e., tab delimited files) from nmdc zip file
The NMDC data is currently stored in a zip file. Instead of unzipping the file, simply use the `zipfile` library to load the `study`, `project`, `contact`, `project_biosample`, and `biosample` tables (stored as tab-delimited files). The `project_biosample` table is needed as a cross-linking table between `project` and `biosample`. The `contact` table contains information about principal investigators.

The code for unzipping and creating the dataframe is found in the `make_dataframe` function. As part of the dataframe creation process, the column names are lower cased and spaces are replaced with underscored. I find it helpful to have some standarization on column names when doing data wrangling. This behavior can be overridden if you wish.

In [2]:
study = dop.make_dataframe("export.sql/STUDY_DATA_TABLE.dsv", file_archive_name="data/nmdc-version2.zip")
contact = dop.make_dataframe("export.sql/CONTACT_DATA_TABLE.dsv", file_archive_name="data/nmdc-version2.zip")
project = dop.make_dataframe("export.sql/PROJECT_DATA_TABLE.dsv", file_archive_name="data/nmdc-version2.zip")
project_biosample = dop.make_dataframe("export.sql/PROJECT_BIOSAMPLE_DATA_TABLE.dsv", file_archive_name="data/nmdc-version2.zip")
biosample = dop.make_dataframe("export.sql/BIOSAMPLE_DATA_TABLE.dsv", file_archive_name="data/nmdc-version2.zip")
proposals = dop.make_dataframe("data/JGI-EMSL-FICUS-proposals.fnl.tsv")

## Subset tables to records where active = 'Yes'

In [3]:
q = """
select 
    *
from
    study
where
    active = 'Yes'
"""
study = sqldf(q)

In [4]:
q = """
select 
    *
from
    project
where
    active = 'Yes'
"""
project = sqldf(q)

In [5]:
q = """
select 
    *
from
    biosample
where
    active = 'Yes'
"""
biosample = sqldf(q)

In [6]:
# biosample.head() # peek at data

## Build json data files
The json data files are build using a general approach:
1. Create a pandas dataframe (often using SQL syntax) to be translated.
2. Transform the dataframe into a dictionary (these variables end with '_dictdf')
3. Define a list of field names whose names and values will be translated into characteristics within an annotation object.
4. Pass the dataframe dictionary and characteristices list to the `make_json_string_list` method. This method returns a list of json ojbects each of which has been converted to a string.
5. Save the json string to file using `save_json_string_list`.

**Note:** Currently, I am using the GOLD IDs as idenifiers. This need to changed to IRIs.

## Build study json
* Create a subset of the study table using the FICUS gold_ids specified in [JGI-EMSL-FICUS-proposals.fnl.xlxs](https://docs.google.com/spreadsheets/d/1sowTCYooDrOMq0ErD4s3xtgH3PLoxwa7/edit#gid=1363834365).
* Follow approach for building json data files.

In [7]:
q = """
select 
    study.*, contact.name as principal_investigator_name, proposals.doi
from
    study
left join
    contact
on
    study.contact_id = contact.contact_id
left join
    proposals
on
    study.gold_id = proposals.gold_study
where
    study.gold_id in 
      ('Gs0110115', 'Gs0110132', 'Gs0112340', 'Gs0114675', 'Gs0128849', 'Gs0130354', 
       'Gs0114298', 'Gs0114663', 'Gs0120351', 'Gs0134277', 'Gs0133461', 'Gs0135152', 'Gs0135149')
"""
study = sqldf(q)

In [8]:
study.head() # peek at data

,study_id,study_name,its_proposal_id,ncbi_project_id,add_date,mod_date,contact_id,last_mod_by,is_public,gpts_proposal_id,...,public_dap_count,admin_dap_count,public_biosample_count,admin_biosample_count,is_geba,is_hmp,ecosystem_path_id,is_test,principal_investigator_name,doi
0,114663,Groundwater microbial communities from the Col...,NaN,None,12-MAY-15 12.00.00.000000000 AM,08-JAN-16 12.00.00.000000000 AM,113389,102151.0,Yes,None,...,0,7,50,50,No,No,4165,No,James Stegen,10.25585/1487765
1,114298,Permafrost soil microbial communities from the...,1782.0,None,05-FEB-15 12.00.00.000000000 AM,None,102898,NaN,Yes,None,...,0,1,24,24,No,No,4234,No,Byron Crump,10.25585/1487766
2,110115,Avena fatua rhizosphere microbial communities ...,1472.0,None,11-NOV-13 12.00.00.000000000 AM,29-MAY-14 12.00.00.000000000 AM,108484,103865.0,Yes,None,...,0,0,26,26,No,No,3948,No,Mary Firestone,10.25585/1487760
3,110132,Cyanobacterial communities from the Joint Geno...,1393.0,None,13-DEC-13 12.00.00.000000000 AM,07-FEB-14 12.00.00.000000000 AM,100043,103865.0,Yes,None,...,0,13,8,8,No,No,4318,No,Matthias Hess,10.25585/1487758
4,112340,Thawing permafrost microbial communities from ...,1778.0,None,24-OCT-14 12.00.00.000000000 AM,31-OCT-14 12.00.00.000000000 AM,110735,105383.0,Yes,None,...,0,0,6,6,No,No,4206,No,Virginia Rich,10.25585/1487764


In [9]:
study_dictdf = study.to_dict(orient="records") # transorm dataframe to dictionary

In [10]:
## print out a single record for viewing
# for record in study_dictdf:
#     print(json.dumps(record, indent=4)); break

In [11]:
## specify characteristics
characteristics = \
    ['gold_study_name', 'principal_investigator_name', 'add_date', 'mod_date', 'doi',
      'ecosystem', 'ecosystem_category', 'ecosystem_type', 'ecosystem_subtype', 'specific_ecosystem', 'ecosystem_path_id']

## create list of json string objects
study_json_list = dop.make_json_string_list\
    (study_dictdf, nmdc.Study, id_key='gold_id', name_key='study_name', description_key="description", characteristic_fields=characteristics)

In [12]:
print(study_json_list[0]) ## peek at data

{
   "id": "Gs0114663",
   "name": "Groundwater microbial communities from the Columbia River, Washington, USA",
   "description": "A metagenomic study to couple microbial communities to carbon and contaminant biogeochemistry in the groundwater-surface water interaction zone",
   "annotations": [
      {
         "has_characteristic": {
            "name": "add_date"
         },
         "has_raw_value": "12-MAY-15 12.00.00.000000000 AM"
      },
      {
         "has_characteristic": {
            "name": "mod_date"
         },
         "has_raw_value": "08-JAN-16 12.00.00.000000000 AM"
      },
      {
         "has_characteristic": {
            "name": "gold_study_name"
         },
         "has_raw_value": "Microbial communities from the groundwater-surface water interaction zone near Columbia River, Washington, USA and their role in carbon and contaminant biogeochemistry"
      },
      {
         "has_characteristic": {
            "name": "ecosystem"
         },
         "has_r

In [13]:
dop.save_json_string_list("output/nmdc-json/study.json", study_json_list) # save json string list to file

## Buid project json
* Create dataframe for projects that are part of the FICUS studies.
* Follow approach for building json data files.

In [14]:
q = """
select
    project.gold_id, project.project_name, project.description, project.add_date, 
    project.mod_date, project.completion_date, project.ncbi_project_name, study.gold_id as study_gold_id
from 
    project
inner join study
    on study.study_id = project.master_study_id
"""
projectdf = sqldf(q)

In [15]:
project_dictdf = projectdf.to_dict(orient="records") # transorm dataframe to dictionary

In [16]:
## specify characteristics
characteristics = \
    ['add_date', 'mod_date', 'completion_date', 'ncbi_project_name']

## create list of json string objects
project_json_list = dop.make_json_string_list\
    (project_dictdf, nmdc.SequencingProject, id_key='gold_id', name_key='project_name', 
     part_of_key="study_gold_id", description_key="description", characteristic_fields=characteristics)

In [17]:
# print(project_json_list[0]) ## peek at data

In [18]:
dop.save_json_string_list("output/nmdc-json/project.json", project_json_list) # save json string list to file

## Build biosample json
* Create dataframe for biosamples that are part of the FICUS studies. Note the use of `group_concat` in the query string. This is needed b/c a biosample may potentially belong to more than one project.
* Follow approach for building json data files.

**Note:** The list of characteristics is quite long. I might need to rething a more elegant way to do this.

In [19]:
q = """
select
    biosample.gold_id,
    biosample.biosample_name,
    biosample.description,
    biosample.add_date,
    biosample.mod_date,
    biosample.ecosystem_path_id,
    biosample.ecosystem,
    biosample.ecosystem_category,
    biosample.ecosystem_type,
    biosample.ecosystem_subtype,
    biosample.specific_ecosystem,
    biosample.habitat,
    biosample.location,
    biosample.community,
    biosample.ncbi_taxonomy_name,
    biosample.geographic_location,
    biosample.latitude,
    biosample.longitude,
    biosample.sample_collection_site,
    biosample.identifier,
    biosample.sample_collection_year,
    biosample.sample_collection_month,
    biosample.sample_collection_day,
    biosample.sample_collection_hour,
    biosample.sample_collection_minute,
    biosample.host_name,
    biosample.depth,
    biosample.subsurface_depth,
    biosample.altitude,
    biosample.temperature_range,
    biosample.proport_woa_temperature,
    biosample.biogas_temperature,
    biosample.growth_temperature,
    biosample.soil_annual_season_temp,
    biosample.water_samp_store_temp,
    biosample.biogas_retention_time,
    biosample.salinity,
    biosample.pressure,
    biosample.ph,
    biosample.chlorophyll_concentration,
    biosample.nitrate_concentration,
    biosample.oxygen_concentration,
    biosample.salinity_concentration,
    group_concat(project.gold_id) as project_gold_ids
from
    biosample
inner join project_biosample
    on biosample.biosample_id = project_biosample.biosample_id
inner join project
    on project.project_id = project_biosample.project_id
group by
    biosample.biosample_id,
    biosample.biosample_name,
    biosample.description,
    biosample.add_date,
    biosample.mod_date,
    biosample.ecosystem_path_id,
    biosample.ecosystem,
    biosample.ecosystem_category,
    biosample.ecosystem_type,
    biosample.ecosystem_subtype,
    biosample.specific_ecosystem,
    biosample.habitat,
    biosample.location,
    biosample.community,
    biosample.ncbi_taxonomy_name,
    biosample.geographic_location,
    biosample.latitude,
    biosample.longitude,
    biosample.sample_collection_site,
    biosample.identifier,
    biosample.sample_collection_year,
    biosample.sample_collection_month,
    biosample.sample_collection_day,
    biosample.sample_collection_hour,
    biosample.sample_collection_minute,
    biosample.host_name,
    biosample.depth,
    biosample.subsurface_depth,
    biosample.altitude,
    biosample.temperature_range,
    biosample.proport_woa_temperature,
    biosample.biogas_temperature,
    biosample.growth_temperature,
    biosample.soil_annual_season_temp,
    biosample.water_samp_store_temp,
    biosample.biogas_retention_time,
    biosample.salinity,
    biosample.pressure,
    biosample.ph,
    biosample.chlorophyll_concentration,
    biosample.nitrate_concentration,
    biosample.oxygen_concentration,
    biosample.salinity_concentration
"""
biosampledf = sqldf(q)

In [20]:
biosample_dictdf = biosampledf.to_dict(orient="records") # transorm dataframe to dictionary

In [21]:
## specify characteristics
characteristics = \
    ['add_date',
     'mod_date',
     'ecosystem_path_id',
     'ecosystem',
     'ecosystem_category',
     'ecosystem_type',
     'ecosystem_subtype',
     'specific_ecosystem',
     'habitat',
     'location',
     'community',
     'ncbi_taxonomy_name',
     'geographic_location',
     'latitude',
     'longitude',
     'sample_collection_site',
     'identifier',
     'sample_collection_year',
     'sample_collection_month',
     'sample_collection_day',
     'sample_collection_hour',
     'sample_collection_minute',
     'host_name',
     'depth',
     'subsurface_depth',
     'altitude',
     'temperature_range',
     'proport_woa_temperature',
     'biogas_temperature',
     'growth_temperature',
     'soil_annual_season_temp',
     'water_samp_store_temp',
     'biogas_retention_time',
     'salinity',
     'pressure',
     'ph',
     'chlorophyll_concentration',
     'nitrate_concentration',
     'oxygen_concentration',
     'salinity_concentration'
    ]

In [22]:
## create list of json string objects
biosample_json_list = dop.make_json_string_list\
    (biosample_dictdf, nmdc.Biosample, id_key='gold_id', name_key='biosample_name', 
     part_of_key="project_gold_ids", description_key="description", characteristic_fields=characteristics)

In [23]:
# print(biosample_json_list[0]) # peek at data

In [24]:
dop.save_json_string_list("output/nmdc-json/biosample.json", biosample_json_list) # save json string list to file